In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.insert(0, '../../')
import src.results as results
import pandas as pd
from math import sqrt

d:\ProgramData\Anaconda3\envs\rwn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def agg_res(res, name=None):
    new_res = {'classification':{}, 'link_pred':{}}

    for agg, runs in res.items():
        new_res[agg] = {}
        classification = []
        link_pred = []
        for r in runs:
            classification.append(r[0][2])
            link_pred.append(float(r[0][1].cpu().numpy()))
        new_res['classification'][agg] = classification
        new_res['link_pred'][agg] = link_pred

    if name is not None:
        print('=' * 50, f' {name.upper()} ', '='*50)
    print('Classification Acc')
    df = pd.DataFrame(new_res['classification'])
    display(df.mean())
    print()
    display(df.std())

    print('-'*100)

    print('Link Pred AUC')
    df = pd.DataFrame(new_res['link_pred'])
    display(df.mean())
    print()
    display(df.std())
    print()
    print()

    return new_res


In [5]:
funcs = [
    lambda x, y: 0.5,  # 0 
    lambda x, y: 0.9,  # 1
    lambda x, y: 0.1,  # 2
    lambda x, y: 0.25,  # 3
    lambda x, y: 0.75,  # 4
    lambda x, y: x / y,  # 5

    # These next three have no rationale but fuck it who cares
    lambda x, y: (x / y) ** 2,
    lambda x, y: sqrt(x / y),
    lambda x, y: y / x
    ]

In [5]:
res = []
for i, l in enumerate(funcs):
    x = results.run_joint_train('Cora', 3, epochs=1000, lamb=l)
    res.append(agg_res(x, name=f'Lambda #{i}'))

100%|██████████| 3/3 [01:01<00:00, 20.66s/it]

==================================================  LAMBDA #0  ==================================================
Classification Acc


eve     0.693270
mean    0.824131
gcn     0.889204
pool    0.815684
dtype: float64

eve     0.192849
mean    0.002011
gcn     0.016488
pool    0.007536
dtype: float64

----------------------------------------------------------------------------------------------------
Link Pred AUC


...

eve     0.480556
mean    0.458999
gcn     0.481751
pool    0.538447
dtype: float64

eve     0.061350
mean    0.002349
gcn     0.002858
pool    0.001548
dtype: float64

----------------------------------------------------------------------------------------------------
Link Pred AUC


eve     0.110333
mean    0.098333
gcn     0.105333
pool    0.077000
dtype: float64

eve     0.033486
mean    0.026858
gcn     0.042548
pool    0.022517
dtype: float64

In [6]:

res_pubmed = []
for i, l in enumerate(funcs):
    x = results.run_joint_train('PubMed', 3, epochs=1000, lamb=l)
    res_pubmed.append(agg_res(x, name=f'Lambda #{i}'))

100%|██████████| 3/3 [01:16<00:00, 25.54s/it]

==================================================  LAMBDA #0  ==================================================
Classification Acc


eve     0.566332
mean    0.896620
gcn     0.892837
pool    0.886450
dtype: float64

eve     0.180895
mean    0.003692
gcn     0.000860
pool    0.009755
dtype: float64

----------------------------------------------------------------------------------------------------
Link Pred AUC


...

eve     0.490039
mean    0.411692
gcn     0.435267
pool    0.617026
dtype: float64

eve     0.070107
mean    0.002353
gcn     0.016826
pool    0.003053
dtype: float64

----------------------------------------------------------------------------------------------------
Link Pred AUC


eve     0.255667
mean    0.388667
gcn     0.333333
pool    0.333333
dtype: float64

eve     0.131059
mean    0.002887
gcn     0.132824
pool    0.132824
dtype: float64

In [7]:
res_citeseer = []
for i, l in enumerate(funcs):
    x = results.run_joint_train('CiteSeer', 3, epochs=1000, lamb=l)
    res_citeseer.append(agg_res(x, name=f'Lambda #{i}'))

100%|██████████| 3/3 [03:33<00:00, 71.01s/it]

==================================================  LAMBDA #0  ==================================================
Classification Acc


eve     0.531471
mean    0.815377
gcn     0.872618
pool    0.772899
dtype: float64

eve     0.129804
mean    0.008498
gcn     0.006984
pool    0.008662
dtype: float64

----------------------------------------------------------------------------------------------------
Link Pred AUC


...

eve     0.514639
mean    0.345779
gcn     0.427796
pool    0.547975
dtype: float64

eve     0.016192
mean    0.008714
gcn     0.010302
pool    0.001127
dtype: float64

----------------------------------------------------------------------------------------------------
Link Pred AUC


eve     0.170333
mean    0.180667
gcn     0.193667
pool    0.147000
dtype: float64

eve     0.011060
mean    0.012503
gcn     0.032884
pool    0.060622
dtype: float64